In [ ]:
import bz2
import subprocess
import re
import xml.sax
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))
#src: https://towardsdatascience.com/wikipedia-data-science-working-with-the-worlds-largest-encyclopedia-c08efbac5f5c

In [ ]:
# Object for handling xml
handler = WikiXmlHandler()# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)# Iteratively process file
for line in subprocess.Popen(['bzcat'], 
                              stdin = open("/content/drive/My Drive/Colab Notebooks/etwiki-latest-pages-articles.xml.bz2"), 
                              stdout = subprocess.PIPE).stdout:
    parser.feed(line)
#src: https://towardsdatascience.com/wikipedia-data-science-working-with-the-worlds-largest-encyclopedia-c08efbac5f5c

In [ ]:
#make dictionary with all pages and texts
#make list with (disambiguation page, text)-s
disambiguation_pages_texts = []
all_pages ={}
for el in handler._pages:
  if ('{{täpsustus}}' in el[1] or '{{täpsustuslehekülg}}' in el[1]) and el[0] != 'Vikipeedia:Vormistusreeglid':
    disambiguation_pages_texts.append(el)
  all_pages[el[0].lower()]=el[1]

In [ ]:
#make list of (disambiguation page, senses)-s
disambiguation_pages_senses = []
for el in disambiguation_pages_texts:
  senses = []
  v = el[1].split('Vaata ka')[0].split('*')
  for s in v:
    links = re.findall("\[\[(.*?)\]\]",s)
    if links != []:
      senses.append(links[0].split('|')[0])
  disambiguation_pages_senses.append((el[0],senses))

In [ ]:
#function that takes sense and returns (sense, number of sentences)
def a(sense):
  title = sense.lower()
  nr = 0
  if title in all_pages.keys():
    if bool(re.match('#suuna\s*\[\[.*?\]\]\s*',all_pages[title])):
      new_page = re.findall('#suuna\s*\[\[(.*?)\]\]\s*',all_pages[title])[0].lower().strip().replace('_', ' ')
      if new_page in all_pages.keys():
        p = re.compile(r'==\s*Viited\s*==')
        nr = len(sent_tokenize(p.split(all_pages[new_page])[0]))
    else:
      p = re.compile(r'==\s*Viited\s*==')
      nr = len(sent_tokenize(p.split(all_pages[title])[0]))
    return (title,nr)
  return ('',nr)

In [ ]:
#function that takes list of senses and returns list of tuples (sense, number of sentences)
def b(lst):
  lst1 = []
  for el in lst:
    lst1.append(a(el))
  return lst1

In [ ]:
#output will be dict of disambiguation titles with lists of their senses and numbers of sentences for each sense
output = {}
for t,s in disambiguation_pages_senses:
  input = s
  output[t]=b(input)

In [ ]:
#number of disambiguation pages that have more than one senses, including red links
counter = 0
for el in output.keys():
  if len(output[el]) > 1:
    counter += 1
print(counter)

6360


In [ ]:
#example of data, ('', 0)-s represent senses which don't have article about them
sample = random.sample(output.keys(), 10)
for el in sample:
  print(el, output[el])

Suurevälja [('', 0), ('', 0), ('', 0)]
CU [('vask', 70), ('rünkpilv', 16), ('.cu', 4), ('kirikuslaavi keel', 10)]
Dorothea [('dorothea (eesnimi)', 4), ('brandenburgi dorothea', 13)]
Tartu lahing [('tartu lahing (1224)', 96), ('tartu piiramine (1704)', 37), ('tartu lahing (1919)', 27), ('', 0), ('tartu lahing (1944)', 196)]
CC [('creative commons', 26), ('', 0), ('kookossaared', 13), ('helikassett', 102), ('', 0), ('kiudrünkpilved', 14), ('kulon', 5), ('c (programmeerimiskeel)', 20), ('.cc', 4)]
Viru tänav (täpsustus) [('tänav', 13), ('', 0), ('', 0), ('', 0), ('viru tänav (narva)', 16), ('', 0), ('viru tänav (pärnu)', 4), ('', 0), ('viru tänav (rakvere)', 11), ('viru tänav', 53), ('', 0)]
Anatoli Nikolajev (täpsustus) [('', 0), ('anatoli nikolajev', 216), ('', 0), ('anatoli pavlovitš nikolajev', 5), ('', 0), ('', 0)]
Lohu tänav (täpsustus) [('tänav', 13), ('', 0), ('', 0), ('', 0), ('', 0), ('', 0), ('lohu tänav', 8), ('lohu tänav (viljandi)', 3), ('', 0)]
Kasarmu tänav [('kasarmu täna

In [ ]:
#disambiguation pages, included only senses that have article about them
output2 = output
minus = []
for el in output2.keys():
    x = output2[el]
    output2[el]=list(filter(lambda a: a != ('', 0), x)) #src: https://stackoverflow.com/questions/1157106/remove-all-occurrences-of-a-value-from-a-list
    if output2[el]==[]:
        minus.append(el)
for e in minus:
    output2.pop(e)

In [ ]:
#number of disambiguation pages that have more than one senses, excluding red links
counter = 0
for el in output2.keys():
  if len(output2[el]) > 1:
    counter += 1
print(counter)

4512


In [ ]:
#example of data
for el in sample:
  if el in output2.keys():
    print(el, output2[el])

CU [('vask', 70), ('rünkpilv', 16), ('.cu', 4), ('kirikuslaavi keel', 10)]
Dorothea [('dorothea (eesnimi)', 4), ('brandenburgi dorothea', 13)]
Tartu lahing [('tartu lahing (1224)', 96), ('tartu piiramine (1704)', 37), ('tartu lahing (1919)', 27), ('tartu lahing (1944)', 196)]
CC [('creative commons', 26), ('kookossaared', 13), ('helikassett', 102), ('kiudrünkpilved', 14), ('kulon', 5), ('c (programmeerimiskeel)', 20), ('.cc', 4)]
Viru tänav (täpsustus) [('tänav', 13), ('viru tänav (narva)', 16), ('viru tänav (pärnu)', 4), ('viru tänav (rakvere)', 11), ('viru tänav', 53)]
Anatoli Nikolajev (täpsustus) [('anatoli nikolajev', 216), ('anatoli pavlovitš nikolajev', 5)]
Lohu tänav (täpsustus) [('tänav', 13), ('lohu tänav', 8), ('lohu tänav (viljandi)', 3)]
Kasarmu tänav [('kasarmu tänav (tartu)', 5), ('kategooria:eesti tänavate täpsustusleheküljed', 2)]
Salu mõis [('karjamõis', 9)]
